In [ ]:
import os

def get_repo_dir():
    cwd = os.getcwd()
    splited = cwd.split('/')
    ind = splited.index('fsCounter')
    repo_dir = '/'
    for s in splited[1:ind + 1]:
        repo_dir = os.path.join(repo_dir, s)

    return repo_dir

In [ ]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display



sys.path.append(get_repo_dir())

from vision.tools.jupyter_notebooks.notebook_analysis_help_funcs import *
from vision.tools.post_process_analysis import read_tracks_and_slices, get_block_count
from vision.visualization.draw_bb_from_csv import draw_tree_bb_from_tracks

In [ ]:
def concat_to_meta(block_meta, df):
    df_col = list(block_meta.columns)
    new_data = []
    data = block_meta.copy()
    for id_, sample in data.iterrows():
        block = sample['block'].lower()
        row = sample['row'].lower()
        tree_id = int(sample['tree_id'])
        q_data = df.query(f'block == "{block}" and row == "{row}" and tree_id == {tree_id}')
        new_sample = sample.to_list()
        for i in range(1, 4):
            new_sample.append(q_data[str(i)].values[0])
        new_data.append(new_sample)

    df_col += ['cv1', 'cv2', 'cv3']
    new_df = pd.DataFrame(new_data, columns=df_col)

    return new_df

def add_ratios(df):
    df['F/cv1'] = df['F'] / df['cv1']
    df['F/cv2'] = df['F'] / df['cv2']
    df['F/cv3'] = df['F'] / df['cv3']

    return df

def get_block_ratio(block_df, row_tracks, y_threshold=800, depth=3):
    block_col = list(block_df.columns)
    new_data = []
    for id_, sample in block_df.iterrows():
        row = sample['row'].lower()
        tree_id = int(sample['tree_id'])
        tree_df = row_tracks[row][tree_id]

        d_tree_df = tree_df.query(f'depth <= {depth}')
        
        
        lower_tree_df = tree_df.query(f'y1 > {y_threshold} and depth <= {depth}')
        count = len(tree_df.track_id.unique())
        lower_count = len(lower_tree_df.track_id.unique())
        ratio = lower_count / count

        gdf = lower_tree_df.groupby('track_id')
        lower_tracks_depth = np.array(gdf.depth.mean())
        filtered_lower_tracks_depth = lower_tracks_depth[lower_tracks_depth < 3]
        mean = np.mean(filtered_lower_tracks_depth)
        std = np.std(filtered_lower_tracks_depth)          
        
        new_sample = sample.to_list()
 
        uniq, counts = np.unique(d_tree_df["track_id"], return_counts=True)
        for i in range(0, 5):
            new_sample.append(len(uniq[counts>i]))
            
        new_data.append(new_sample)

    #block_col += ['y_ratio', 'mean', 'std', 'lcv1', 'lcv2', 'lcv3', 'lcv4', 'lcv5']
    block_col += ['dcv1', 'dcv2', 'dcv3', 'dcv4', 'dcv5']
    new_df = pd.DataFrame(new_data, columns=block_col)

    return new_df    

In [ ]:
def linear_model_selection(data, selection_cols=["cv1"], type_col="block", cross_val='row'):

    factors = {}
    for col in selection_cols:
        factor, res_mean, res_std, tree_mean, tree_std, all_preds = run_LROCV(data, cv_col=col, type_col=type_col, cross_val=cross_val, return_res=True)
        factors[col] = {'factor': factor, 'mean_error': res_mean, 'std_error': res_std}

    return factors
    

In [ ]:
def block_analysis(block_path, metadata_path, block_):
    block_counts, row_tracks = get_block_count(block_path)
    block_counts_df = pd.DataFrame(block_counts, columns=['tree_id', 'block', 'row', '1', '2','3'])
    meta_data = pd.read_csv(metadata_path)
    block_meta = meta_data.query(f'block == "{block_}"')
    block_df = concat_to_meta(block_meta, block_counts_df)
    block_df['F/cv1'] = block_df['F'] / block_df['cv1'] 
    block_df = get_block_ratio(block_df, row_tracks)

    return block_df, row_tracks

In [ ]:
def get_selection_error(factors_dict, block_df):
    results = {}
    for item_ in list(factors_dict.keys()):
        block_df[f'err_{item_}'] = (block_df['F'] - (block_df[item_] * factors_dict[item_]['factor'])) / block_df['F']
        results[item_] = {'err': np.mean(block_df[f'err_{item_}']), 'err_std': np.std(block_df[f'err_{item_}'])}

    return results, block_df

In [ ]:
metadata_path = "/media/matans/My Book/FruitSpec/Apples_SA/data_meta.csv"

In [ ]:
block_path = "/media/matans/My Book/FruitSpec/Apples_SA/block_13"
block_ = 'Block_13'
block_13_df, row_tracks_13 = block_analysis(block_path, metadata_path, block_)

In [ ]:
block_13_df

In [ ]:
#row_to_drow = 'row_8888'
#tree_id = 2
#draw_tree_bb_from_tracks(row_tracks[row_to_drow][tree_id], os.path.join(block_path, row_to_drow, '1'), tree_id)

In [ ]:
plt.bar(block_13_df['id'], block_13_df['F/cv1'])
plt.xlabel('Tree')
plt.ylabel('F/CV1')

In [ ]:
plot_F_cv(block_13_df, 1, hue='variety', add_xy_line=False)

In [ ]:
plot_F_cv(block_13_df, 1, add_xy_line=False)

In [ ]:
factors_dict = linear_model_selection(block_13_df, selection_cols=['cv1','dcv1', 'cv3', 'dcv3'],type_col="block", cross_val='id')

In [ ]:
factors_dict

In [ ]:
block_13_factor_cv1 = 0.9917
block_13_factor_dcv1 = 1.00831
block_13_factor_cv3 = 2.0165
block_13_factor_dcv3 = 2.21416

In [ ]:
block_13_df['acc1'] = (block_13_df['F'] - (block_13_df['cv1'] * block_13_factor)) / block_13_df['F']
block_13_df['dacc1'] = (block_13_df['F'] - (block_13_df['dcv1'] * block_13_factor_dcv1)) / block_13_df['F']
block_13_df['acc3'] = (block_13_df['F'] - (block_13_df['cv3'] * block_13_factor_cv3)) / block_13_df['F']
block_13_df['dacc3'] = (block_13_df['F'] - (block_13_df['dcv3'] * block_13_factor_dcv3)) / block_13_df['F']


In [ ]:
block_13_df

In [ ]:
block_mean_cv1 = np.mean(block_13_df['acc1'])
block_std_cv1 = np.std(block_13_df['acc1'])
block_mean_dcv1 = np.mean(block_13_df['dacc1'])
block_std_dcv1 = np.std(block_13_df['dacc1'])
block_mean_cv3 = np.mean(block_13_df['acc3'])
block_std_cv3 = np.std(block_13_df['acc3'])
block_mean_dcv3 = np.mean(block_13_df['dacc3'])
block_std_dcv3 = np.std(block_13_df['dacc3'])
print(f'CV1 block ACC: {block_mean_cv1}, block ACC STD: {block_std_cv1}')
print(f'dCV1 block ACC: {block_mean_dcv1}, block ACC STD: {block_std_dcv1}')
print(f'CV3 block ACC: {block_mean_cv3}, block ACC STD: {block_std_cv3}')
print(f'dCV3 block ACC: {block_mean_dcv3}, block ACC STD: {block_std_dcv3}')

In [ ]:
block_21_path = "/media/matans/My Book/FruitSpec/Apples_SA/block_21/block_21"
block_21 = 'Block_21'
block_21_df, row_tracks_21 = block_analysis(block_21_path, metadata_path, block_21)

In [ ]:
block_21_df

In [ ]:
plot_F_cv(block_21_df, 1, add_xy_line=False)

In [ ]:
factors_21_dict = linear_model_selection(block_21_df, selection_cols=['cv1','dcv1', 'cv3', 'dcv3'],type_col="block", cross_val='id')

In [ ]:
#row_to_drow = 'row_1111'
#tree_id = 1
#draw_tree_bb_from_tracks(row_tracks[row_to_drow][tree_id], os.path.join(block_path, row_to_drow, '1'), tree_id)

In [ ]:
block_21_factor_cv1 = 1.599476
block_21_factor_dcv1 = 1.66923088
block_21_factor_cv3 = 3.08106144
block_21_factor_dcv3 = 3.27543787

block_21_df['acc1'] = (block_21_df['F'] - (block_21_df['cv1'] * block_21_factor_cv1)) / block_21_df['F']
block_21_df['dacc1'] = (block_21_df['F'] - (block_21_df['dcv1'] * block_21_factor_dcv1)) / block_21_df['F']
block_21_df['acc3'] = (block_21_df['F'] - (block_21_df['cv3'] * block_21_factor_cv3)) / block_21_df['F']
block_21_df['dacc3'] = (block_21_df['F'] - (block_21_df['dcv3'] * block_21_factor_dcv3)) / block_21_df['F']

block_mean_cv1 = np.mean(block_21_df['acc1'])
block_std_cv1 = np.std(block_21_df['acc1'])
block_mean_dcv1 = np.mean(block_21_df['dacc1'])
block_std_dcv1 = np.std(block_21_df['dacc1'])
block_mean_cv3 = np.mean(block_21_df['acc3'])
block_std_cv3 = np.std(block_21_df['acc3'])
block_mean_dcv3 = np.mean(block_21_df['dacc3'])
block_std_dcv3 = np.std(block_21_df['dacc3'])
print(f'CV1 block ACC: {block_mean_cv1}, block ACC STD: {block_std_cv1}')
print(f'dCV1 block ACC: {block_mean_dcv1}, block ACC STD: {block_std_dcv1}')
print(f'CV3 block ACC: {block_mean_cv3}, block ACC STD: {block_std_cv3}')
print(f'dCV3 block ACC: {block_mean_dcv3}, block ACC STD: {block_std_dcv3}')

In [ ]:
plt.bar(block_21_df['id'], block_21_df['F/cv1'])
plt.xlabel('Tree')
plt.ylabel('F/CV1')

In [ ]:
block_48_path = "/media/matans/My Book/FruitSpec/Apples_SA/block_48/block_48"
block_48 = 'Block_48'
block_48_df, row_tracks_48 = block_analysis(block_48_path, metadata_path, block_48)

In [ ]:
block_48_df

In [ ]:
factors_48_dict = linear_model_selection(block_48_df, selection_cols=['cv1','dcv1', 'cv3', 'dcv3'],type_col="block", cross_val='id')

In [ ]:
block_48_df['acc1'] = (block_48_df['F'] - (block_48_df['cv1'] * factors_48_dict['cv1']['factor'])) / block_48_df['F']
block_48_df['dacc1'] = (block_48_df['F'] - (block_48_df['dcv1'] * factors_48_dict['dcv1']['factor'])) / block_48_df['F']
block_48_df['acc3'] = (block_48_df['F'] - (block_48_df['cv3'] * factors_48_dict['cv3']['factor'])) / block_48_df['F']
block_48_df['dacc3'] = (block_48_df['F'] - (block_48_df['dcv3'] * factors_48_dict['dcv3']['factor'])) / block_48_df['F']

block_mean_cv1 = np.mean(block_48_df['acc1'])
block_std_cv1 = np.std(block_48_df['acc1'])
block_mean_dcv1 = np.mean(block_48_df['dacc1'])
block_std_dcv1 = np.std(block_48_df['dacc1'])
block_mean_cv3 = np.mean(block_48_df['acc3'])
block_std_cv3 = np.std(block_48_df['acc3'])
block_mean_dcv3 = np.mean(block_48_df['dacc3'])
block_std_dcv3 = np.std(block_48_df['dacc3'])
print(f'CV1 block ACC: {block_mean_cv1}, block ACC STD: {block_std_cv1}')
print(f'dCV1 block ACC: {block_mean_dcv1}, block ACC STD: {block_std_dcv1}')
print(f'CV3 block ACC: {block_mean_cv3}, block ACC STD: {block_std_cv3}')
print(f'dCV3 block ACC: {block_mean_dcv3}, block ACC STD: {block_std_dcv3}')

In [ ]:
factors_48_dict

In [ ]:
plt.bar(block_48_df['id'], block_48_df['F/cv1'])
plt.xlabel('Tree')
plt.ylabel('F/CV1')

In [ ]:
block_3_path = "/media/matans/My Book/FruitSpec/Apples_SA/block_3"
block_3 = 'Block_3'
block_3_df, row_tracks_3 = block_analysis(block_3_path, metadata_path, block_3)

In [ ]:
block_3_df

In [ ]:
factors_3_dict = linear_model_selection(block_3_df, selection_cols=['cv1','dcv1', 'cv3', 'dcv3'],type_col="block", cross_val='id')

In [ ]:
plt.bar(block_3_df['id'], block_3_df['F/cv1'])
plt.xlabel('Tree')
plt.ylabel('F/CV1')

In [ ]:
res, block_3_df_res = get_selection_error(factors_3_dict, block_3_df)

In [ ]:
res

In [ ]:
block_3_df_res